##### **** These pip install need to be adapted to use the appropriate release level. Currently used for testing alpha release of transforms 1.0

In [1]:
%%capture
!pip install 'data-prep-toolkit-transforms[language]==1.0.0a0'
import pyarrow.parquet as pq
import pandas as pd

In [2]:
# Must enable nested asynchronous io in a notebook as the crawler uses coroutine to speed up acquisition and downloads
import nest_asyncio
nest_asyncio.apply()

In [3]:
%%capture
from dpk_web2parquet.transform import Web2Parquet
Web2Parquet(urls= ['https://arxiv.org/pdf/2408.09869'],
                    depth=2, 
                    downloads=10,
                    folder='downloads').transform()


In [4]:
##### **** The specified downloads folder will include the downloaded file(s).
#import glob
#glob.glob("downloads/*") 

In [5]:
from dpk_pdf2parquet.transform_python import Pdf2Parquet
Pdf2Parquet(input_folder= "downloads", 
               output_folder= "pdf2parquet-files", 
               data_files_to_use=['.pdf'],
               pdf2parquet_contents_type='text/markdown').transform()

10:55:10 INFO - pdf2parquet parameters are : {'batch_size': -1, 'artifacts_path': None, 'contents_type': <pdf2parquet_contents_types.MARKDOWN: 'text/markdown'>, 'do_table_structure': True, 'do_ocr': True, 'ocr_engine': <pdf2parquet_ocr_engine.EASYOCR: 'easyocr'>, 'bitmap_area_threshold': 0.05, 'pdf_backend': <pdf2parquet_pdf_backend.DLPARSE_V2: 'dlparse_v2'>, 'double_precision': 8}
10:55:10 INFO - pipeline id pipeline_id
10:55:10 INFO - code location None
10:55:10 INFO - data factory data_ is using local data access: input_folder - downloads output_folder - pdf2parquet-files
10:55:10 INFO - data factory data_ max_files -1, n_sample -1
10:55:10 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.pdf'], files to checkpoint ['.parquet']
10:55:10 INFO - orchestrator pdf2parquet started at 2024-12-14 10:55:10
10:55:10 INFO - Number of files is 1, source profile {'max_file_size': 5.308699607849121, 'min_file_size': 5.3086996078

0

In [6]:
##### **** To explote the output from pdf2parquet, run the code below
#table = pq.read_table('pdf2parquet-files/arxiv_org_2408.09869v5.pdf_application.parquet')
#table.to_pandas()

In [7]:
%%capture
from dpk_doc_chunk.transform_python import DocChunk
DocChunk(input_folder='pdf2parquet-files',
        output_folder='doc-chunk-files',
        doc_chunk_chunking_type= "li_markdown").transform()

10:56:09 INFO - pipeline id pipeline_id
10:56:09 INFO - code location None
10:56:09 INFO - data factory data_ is using local data access: input_folder - pdf2parquet-files output_folder - doc-chunk-files
10:56:09 INFO - data factory data_ max_files -1, n_sample -1
10:56:09 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
10:56:09 INFO - orchestrator doc_chunk started at 2024-12-14 10:56:09
10:56:09 INFO - Number of files is 1, source profile {'max_file_size': 0.023062705993652344, 'min_file_size': 0.023062705993652344, 'total_file_size': 0.023062705993652344}
10:56:09 INFO - Completed 1 files (100.0%) in 0.001 min
10:56:09 INFO - Done processing 1 files, waiting for flush() completion.
10:56:09 INFO - done flushing in 0.0 sec
10:56:09 INFO - Completed execution in 0.001 min, execution result 0


In [9]:
##### **** To explote the output from doc-chunk, run the code below
#table = pq.read_table('doc-chunk-files/arxiv_org_2408.09869v5.pdf_application.parquet')
#table.to_pandas()

In [10]:
from dpk_ededup.transform_python import Ededup
Ededup(input_folder="doc-chunk-files",
    output_folder="dedup-files",
    ededup_doc_column="contents",
    ededup_doc_id_column="document_id").transform()

10:56:59 INFO - exact dedup params are {'doc_column': 'contents', 'doc_id_column': 'document_id', 'use_snapshot': False, 'snapshot_directory': None}
10:56:59 INFO - pipeline id pipeline_id
10:56:59 INFO - code location None
10:56:59 INFO - data factory data_ is using local data access: input_folder - doc-chunk-files output_folder - dedup-files
10:56:59 INFO - data factory data_ max_files -1, n_sample -1
10:56:59 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
10:56:59 INFO - orchestrator ededup started at 2024-12-14 10:56:59
10:56:59 INFO - Number of files is 1, source profile {'max_file_size': 0.03043651580810547, 'min_file_size': 0.03043651580810547, 'total_file_size': 0.03043651580810547}
10:56:59 INFO - Starting from the beginning
10:56:59 INFO - Completed 1 files (100.0%) in 0.0 min
10:56:59 INFO - Done processing 1 files, waiting for flush() completion.
10:56:59 INFO -

0

In [11]:
##### **** To explote the output from eDedup, run the code below
#table = pq.read_table('dedup-files/arxiv_org_2408.09869v5.pdf_application.parquet')
#table.to_pandas()

In [12]:
from dpk_lang_id.transform_python import LangId
LangId(input_folder= "dedup-files",
        output_folder= "langId-files",
        lang_id_model_credential= "PUT YOUR OWN HUGGINGFACE CREDENTIAL",
        lang_id_model_kind= "fasttext",
        lang_id_model_url= "facebook/fasttext-language-identification",
        lang_id_content_column_name= "contents").transform()

10:57:06 INFO - lang_id parameters are : {'model_credential': 'PUT YOUR OWN HUGGINGFACE CREDENTIAL', 'model_kind': 'fasttext', 'model_url': 'facebook/fasttext-language-identification', 'content_column_name': 'contents', 'output_lang_column_name': 'lang', 'output_score_column_name': 'score'}
10:57:06 INFO - pipeline id pipeline_id
10:57:06 INFO - code location None
10:57:06 INFO - data factory data_ is using local data access: input_folder - dedup-files output_folder - langId-files
10:57:06 INFO - data factory data_ max_files -1, n_sample -1
10:57:06 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
10:57:06 INFO - orchestrator lang_id started at 2024-12-14 10:57:06
10:57:06 INFO - Number of files is 1, source profile {'max_file_size': 0.031200408935546875, 'min_file_size': 0.031200408935546875, 'total_file_size': 0.031200408935546875}
10:57:08 INFO - Completed 1 files (100.0%)

0

In [13]:
##### **** To explote the output from langId, run the code below
#table = pq.read_table('langId-files/arxiv_org_2408.09869v5.pdf_application.parquet')
#table.to_pandas()

In [14]:
%%capture
from dpk_doc_quality.transform_python import DocQuality
DocQuality(input_folder='dedup-files',
            output_folder= 'doc-quality-files',
            docq_text_lang = "en",
            docq_doc_content_column ="contents").transform()

10:57:13 INFO - pipeline id pipeline_id
10:57:13 INFO - code location None
10:57:13 INFO - data factory data_ is using local data access: input_folder - dedup-files output_folder - doc-quality-files
10:57:13 INFO - data factory data_ max_files -1, n_sample -1
10:57:13 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
10:57:13 INFO - orchestrator docq started at 2024-12-14 10:57:13
10:57:13 INFO - Number of files is 1, source profile {'max_file_size': 0.031200408935546875, 'min_file_size': 0.031200408935546875, 'total_file_size': 0.031200408935546875}
10:57:13 INFO - Completed 1 files (100.0%) in 0.003 min
10:57:13 INFO - Done processing 1 files, waiting for flush() completion.
10:57:13 INFO - done flushing in 0.0 sec
10:57:13 INFO - Completed execution in 0.003 min, execution result 0


In [15]:
##### **** To explote the output from Doc Quality, run the code below
#table = pq.read_table('doc-quality-files/arxiv_org_2408.09869v5.pdf_application.parquet')
#table.to_pandas()